# Asimétrico: DSA

In [1]:
# !pip install cryptography

In [2]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature
import base64
import time
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Generación de claves DSA

In [ ]:
def generate_keys():
    # Genera una clave privada
    private_key = dsa.generate_private_key(key_size=2048)
    # Obtiene la clave pública correspondiente
    public_key = private_key.public_key()

    return private_key, public_key


Esta línea obtiene la clave pública correspondiente a la clave privada que se acaba de generar.




### Visualizar claves en un formato legible: <br>
PEM Privacy-Enhanced Mail.
- Formato de archivo de texto para representar claves, certificados y firmas criptográficas.

In [ ]:
# ---------------- Función para mostrar las claves privada en formato PEM ----------------
def show_private_key(private_key):
    # Muestra la clave privada DSA en formato PEM.
    pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()  # Sin contraseña para este ejemplo
    )
    print("\nClave privada (PEM):\n")
    print(pem.decode('utf-8'))

def show_public_key(public_key):
    # Muestra la clave pública DSA en formato PEM.
    # Seguro para compartir.
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    print("\nClave pública (PEM):\n")
    print(pem.decode('utf-8'))


### Encriptación, firma y verificación

Firma un mensaje usando la clave privada DSA.
- private_key: objeto DSAPrivateKey de cryptography.
- message: mensaje (str) a firmar.

Retorna la firma en bytes.

In [ ]:
def sign_message(private_key, message):
    # Validar que el mensaje sea una cadena no vacía
    if not isinstance(message, str):
        raise TypeError("El mensaje debe ser una cadena de texto (str).")
    if not message:
        raise ValueError("El mensaje no puede estar vacío.")

    # Convertir el mensaje a bytes (UTF-8)
    data = message.encode('utf-8')
    # Generar la firma usando SHA-256 como función hash
    signature = private_key.sign(data, hashes.SHA256())
    return signature

Verifica la firma de un mensaje usando la clave pública DSA.
- public_key: objeto DSAPublicKey de cryptography.
- message: mensaje original (str).
- signature: firma (bytes) a verificar.

Imprime el resultado de la verificación.

In [ ]:
def verify_signature(public_key, message, signature):
    # Validar inputs
    if not isinstance(message, str):
        raise TypeError("El mensaje debe ser una cadena de texto (str).")
    if not message:
        raise ValueError("El mensaje no puede estar vacío.")
    if not isinstance(signature, bytes):
        raise TypeError("La firma debe ser un objeto de bytes.")
    if not signature:
        raise ValueError("La firma no puede estar vacía.")

    data = message.encode('utf-8')
    try:
        # Verificar la firma: si es inválida
        public_key.verify(signature, data, hashes.SHA256())
        print("Resultado de verificación: Firma válida. El mensaje no fue alterado.")
    except InvalidSignature:
        print("Resultado de verificación: Firma inválida. La firma no corresponde al mensaje.")
    except Exception as e:
        # Otros errores (por ejemplo, formato de firma incorrecto)
        print(f"Error durante la verificación: {e}")

### Ejecución del algoritmo

In [ ]:
def procesar_archivo(path, word_count):
    print(f"\n=== Procesando archivo de {word_count} palabras ===")

    # 1. Leer archivo
    t0 = time.time()
    with open(path, 'r', encoding='utf-8') as f:
        mensaje = f.read().strip()
    t1 = time.time()
    print(f"✔ Texto leído ({len(mensaje.split())} palabras) — Tiempo: {(t1 - t0)*1000:.2f} ms")

    if not mensaje:
        print("⚠️ El archivo está vacío.")
        return

    # 2. Generar claves DSA
    t0 = time.time()
    private_key, public_key = generate_keys()
    t1 = time.time()
    print(f"✔ Claves generadas — Tiempo: {(t1 - t0)*1000:.2f} ms")

    show_private_key(private_key)
    show_public_key(public_key)

    # 3. Firmar el mensaje
    t0 = time.time()
    firma = sign_message(private_key, mensaje)
    t1 = time.time()
    firma_b64 = base64.b64encode(firma).decode('utf-8')
    print(f"✔ Firma digital generada — Tiempo: {(t1 - t0)*1000:.2f} ms")
    print(f"Firma digital (Base64): {firma_b64[:200]}...")

    # 4. Verificación válida
    print("\n→ Verificando firma con mensaje original...")
    t0 = time.time()
    verify_signature(public_key, mensaje, firma)
    t1 = time.time()
    print(f"⏱ Tiempo de verificación: {(t1 - t0)*1000:.2f} ms")

    # 5. Verificación con mensaje alterado
    mensaje_alterado = mensaje + "X"
    print("\n→ Verificando firma con mensaje alterado...")
    print("Mensaje alterado (primeros 200 caracteres):", mensaje_alterado[:200])
    t0 = time.time()
    verify_signature(public_key, mensaje_alterado, firma)
    t1 = time.time()
    print(f"⏱ Tiempo de verificación con mensaje alterado: {(t1 - t0)*1000:.2f} ms")


Definir el directorio donde están los archivos.

In [ ]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/Archivos"

In [ ]:
sizes = [10, 100, 1000, 10000, 100000, 1000000, 10000000]

for n in sizes:
    path = f"{base_folder}/{n}_palabras.txt"
    try:
        procesar_archivo(path, n)
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {path}")